In [99]:
import polars as pl
from pathlib import Path

#pl.Config.with_columns_kwargs = True

CWD = Path.cwd()
INPUT_FILE = f'{CWD}/New Customer Reporting.xlsx'


# -----------------------------------------------------------------------------------------
# load xl file 
# -----------------------------------------------------------------------------------------

xl_sheets = pl.read_excel(INPUT_FILE, sheet_id=0)
xl_sheets_sheet_names_clean = {k.lower().replace(' ', '_'): v for k,v in xl_sheets.items()}

uk_bank_holidays = xl_sheets_sheet_names_clean[list(xl_sheets_sheet_names_clean.keys())[0]]
new_customers = xl_sheets_sheet_names_clean[list(xl_sheets_sheet_names_clean.keys())[1]]
roi_new_customers = xl_sheets_sheet_names_clean[list(xl_sheets_sheet_names_clean.keys())[-1]]

# uk_bank_holidays.head(5), new_customers.head(5), roi_new_customers.head(5)

# -----------------------------------------------------------------------------------------
# transform uk_bank_holidays and new customers data sets
# -----------------------------------------------------------------------------------------

df_comb = ( uk_bank_holidays.with_columns(pl.col('Year').forward_fill()) 
                                     .drop_nulls()
                                     .with_columns((pl.col('Year') + '-' + pl.col('Date')).str.strptime(pl.Date, '%Y-%d-%b')
                                                   .alias('date'))
                                     .drop(['Year', 'Date']) 
                                     # join new_customers data set
                                     .join(new_customers.select([pl.col('New Customers'), 
                                                                pl.col('Date').str.strptime(pl.Date, '%m-%d-%y').alias('date')]),
                                           on='date',
                                           how='outer')
          )

# create flag for reporting days
df = ( df_comb.select([pl.all(),
                       pl.col('date').dt.weekday().alias('weekday'),
                       pl.col('date').dt.strftime('%a').alias('wk_day_name')])
              .select([pl.all(),
                       ~(pl.col('Bank holiday').is_not_null() | pl.col('weekday').is_in([6,7])).alias('is_reporting_day?')])
               # offseting the new customers for the next reporting day
              .groupby(pl.when(pl.col('is_reporting_day?'))
                         .then(pl.col('date'))
                         .backward_fill()
                         .alias('reported_day'),
                          maintain_order=True)
                    .agg([
                           pl.all().last(),
                           pl.col('New Customers').sum().alias('customers')
                        ]) 
              .drop(['Bank holiday', 'date', 'New Customers', 'weekday', 'wk_day_name', 'is_reporting_day?'])
              # dropping the last row which shows nulls for the dates
              .drop_nulls()
                   
     )

 
df_b = ( df.select([
                      pl.all(),
                     (pl.col('reported_day').last().over(pl.col('reported_day').dt.truncate('1mo')))
                     .alias('last_day'),
                  ])  
           .select([
                     pl.all(),
                     pl.when(pl.col('reported_day') < pl.col('last_day'))
                     .then(pl.col('reported_day').dt.strftime('%B') + '-' + pl.col('reported_day').dt.strftime('%Y'))
                     .otherwise(pl.col('reported_day').dt.offset_by('1mo').dt.strftime('%B') + '-' + 
                                pl.col('reported_day').dt.offset_by('1mo').dt.strftime('%Y') )
                     .alias('reported_month')
                  ])
             # filter out the reported_month of jan-2024
            .filter(~pl.col('reported_month').str.contains('2024', literal=True))
            .drop('last_day')
)

# row_number the reported day for each reported_month
df_b = df_b.with_columns(
                          ( pl.col('reported_day').sort(False).cumcount(False).over('reported_month') + 1 )
                              .alias('row_number')
                        )

# transform and clean the roi data set
df_roi = ( roi_new_customers.rename({'Reporting Month' : 'roi_reporting_month', 
                                   'New Customers' : 'roi_new_customers'})
                            .select([pl.all(),
                                     pl.col('Reporting Date').str.strptime(pl.Date, '%m-%d-%y')
                                       .alias('reported_day')
                                   ])
                            .drop(['Reporting Day', 'Reporting Date'])
         )


# joining df_b with df_roi data to add the customers of existing days, 
# which are reporting days in both cases
# df_comb = df_b.join(df_roi, on='reported_day', how='left')
      

# # retrieving back only days where it's an holiday in uk but not in irlenad
df_non_matched = df_roi.join(df_b, on='reported_day', how='anti')



df_out = ( df_b.join(df_roi, on='reported_day', how='outer') 
             .sort('reported_day')#.filter((pl.col('reported_day').dt.month() == 6) & (pl.col('reported_day').dt.year() == 2022)) 
             # find the next reported date (uk) for each reported day from the ROI set which is an holiday in the UK
             .with_columns([pl.when(pl.col('row_number').is_not_null())                                  # Change 1
                              .then(pl.col('reported_day')).backward_fill()
                              .alias('reported_day'),
                            # pl.col('roi_reporting_month').backward_fill(),                               # Change 2
                            pl.when(pl.col('row_number').is_not_null())                                  # Change 3
                              .then(pl.col('row_number')).backward_fill()
                              .alias('row_number'),
                            pl.col('roi_new_customers').fill_null(0),                                    # Change 4
                            pl.col('customers').fill_null(0),                                            # Change 5
                            pl.col('reported_month').backward_fill().alias('reported_month')             # Change 6
                            ]) 
             .groupby(['reported_month', 'reported_day', 'row_number'])
                     .agg([
                            pl.col('customers').sum(),
                            pl.col('roi_new_customers').sum().alias('roi_customers'),
                            pl.col('roi_reporting_month').max()
                          ])
             .sort('reported_day')
             .with_columns(pl.when(
                                   ( (pl.col('reported_month').str.slice(offset=0, length=3) != pl.col('roi_reporting_month').str.slice(offset=0, length=3))
                                     | pl.col('roi_reporting_month').is_null() ))
                             .then('X')
                             .alias('misaligned_flag'))
       )




# -----------------------------------------------------------------------------------------
# output the data to a csv/parquet file
# -----------------------------------------------------------------------------------------

output_path = Path(f'{CWD}/output')
output_path.mkdir(parents=False, exist_ok=False)

df_out.write_csv(f'{output_path}/output-py-sol.csv', has_header=True, sep=',')


